<a href="https://colab.research.google.com/github/DataSka/deep_learning_challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [104]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis='columns')

print(application_df.shape)
application_df.head()

(34299, 10)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [105]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [106]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(app_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [107]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_type_counts[app_type_counts < 1065].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [108]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [109]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_1 = classification_counts[classification_counts > 1]
print(classification_counts_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [110]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts < 194].index.tolist()
# print(classifications)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [111]:
# Convert categorical data to numeric with `pd.get_dummies`
import pandas as pd
application_df = pd.get_dummies(application_df)
print(application_df.shape)
application_df.head()


(34299, 44)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [112]:
target_name = 'IS_SUCCESSFUL'
y = application_df[target_name]
X = application_df
X = X.drop([target_name], axis='columns').values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# print(X_train)

In [113]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
# hidden_nodes_layer3 = 5

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=43))

# First hidden layer
model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               4400      
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 9501 (37.11 KB)
Trainable params: 9501 (37.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5426 - accuracy: 0.7360
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7369
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7357
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5421 - accuracy: 0.7369
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7376
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5420 - accuracy: 0.7361
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7368
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5413 - accuracy: 0.7375
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7359
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7374

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5608 - accuracy: 0.7310 - 529ms/epoch - 2ms/step
Loss: 0.5607741475105286, Accuracy: 0.7309620976448059


In [114]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 110
hidden_nodes_layer2 = 40


model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=43))

# First hidden layer
model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 110)               4840      
                                                                 
 dense_31 (Dense)            (None, 40)                4440      
                                                                 
 dense_32 (Dense)            (None, 1)                 41        
                                                                 
Total params: 9321 (36.41 KB)
Trainable params: 9321 (36.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [115]:
# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [116]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5729 - accuracy: 0.7196
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7290
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7267
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7285
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5502 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7318
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7325
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accura

In [117]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5627 - accuracy: 0.7314 - 485ms/epoch - 2ms/step
Loss: 0.5626542568206787, Accuracy: 0.7314285635948181


In [118]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.3 MB/s eta 0:00:00


In [144]:
def create_model(hp):
    model = tf.keras.models.Sequential()

    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

     # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=3,
        max_value=20,
        step=2), activation=activation, input_dim=43))

      # Allow kerastuner to decide number of hidden layers and neurons in hidden layers

    for i in range(hp.Int('num_layers', 1, 6)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=3,
            max_value=20,
            step=2),
            activation=activation))

    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return model

In [145]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [146]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 01m 02s]
val_accuracy: 0.7310787439346313

Best val_accuracy So Far: 0.7327113747596741
Total elapsed time: 00h 40m 29s


In [147]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 7, 'num_layers': 2, 'units_0': 7, 'units_1': 3, 'units_2': 1, 'units_3': 5, 'units_4': 9, 'units_5': 9, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0038'}
{'activation': 'sigmoid', 'first_units': 1, 'num_layers': 3, 'units_0': 7, 'units_1': 3, 'units_2': 7, 'units_3': 9, 'units_4': 3, 'units_5': 9, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'sigmoid', 'first_units': 5, 'num_layers': 3, 'units_0': 7, 'units_1': 3, 'units_2': 3, 'units_3': 5, 'units_4': 3, 'units_5': 1, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0011'}


In [148]:
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5572 - accuracy: 0.7327 - 1s/epoch - 4ms/step
Loss: 0.557214617729187, Accuracy: 0.7327113747596741
268/268 - 1s - loss: 0.5744 - accuracy: 0.7321 - 1s/epoch - 4ms/step
Loss: 0.5743685364723206, Accuracy: 0.7321282625198364
268/268 - 2s - loss: 0.5763 - accuracy: 0.7318 - 2s/epoch - 6ms/step
Loss: 0.5763465166091919, Accuracy: 0.7317784428596497


In [151]:
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

{'activation': 'sigmoid',
 'first_units': 1,
 'num_layers': 3,
 'units_0': 7,
 'units_1': 3,
 'units_2': 7,
 'units_3': 9,
 'units_4': 3,
 'units_5': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [154]:
model.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
